In [1]:
import norm_heatmap
import normalize
import tools
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import heatmap_workflow_tools as hwt
import os
from mpl_toolkits.mplot3d import Axes3D
import cv2

In [2]:
# In quotes, put paths to folders containing tiffs for Signal of Interest (SOI) and Normalization signal (NS), respectively
stack_dir = os.path.abspath("Insert Path to SOI")
norm_dir = os.path.abspath("Insert Path to NS")

# What is your signal of interest (SOI) and normalization signal (NS) 
soi = "Name of SOI"
ns = "Name of NS"

# Replace with slice integers or decimal percentages. 'None' for no boundary
zmin, zmax = None, None

# Do you want to multiply the pixels in the front and side views (int)?
z_multiplier = 

# Do you want to save your figures? (boolean)
save_figs = False

# Adjust the lower and upper threshold (int)
lower_thresh = 5 
upper_thresh = 254

SyntaxError: invalid syntax (405889449.py, line 13)

In [ ]:
stack_tiffs = tools.get_files(stack_dir)

In [ ]:
norm_tiffs = tools.get_files(norm_dir)

In [ ]:
zmin, zmax = hwt.process_zmin_zmax(zmin, zmax, stack_tiffs)
print(f"Processed ZMin and ZMax: {zmin}, {zmax}")

In [ ]:
stack_name = os.path.basename(stack_dir)
norm_name = os.path.basename(norm_dir)
base_dir = os.path.basename(os.path.dirname(norm_dir))
out_dir = os.path.abspath(os.path.join(os.path.abspath(''), '..', 'exported_data'))
if not os.path.isdir(out_dir):
    os.mkdir(out_dir)
out_dir = os.path.join(out_dir, f"{base_dir}_{stack_name}n{norm_name}_out")
if not os.path.isdir(out_dir):
    os.mkdir(out_dir)

# Determine Images Above Threshold

In [ ]:
norm_imgs = normalize.analyze_images(norm_dir, lower_thresh)
norm_imgs = normalize.images_used(norm_imgs, norm_tiffs, stack_tiffs)

In [ ]:
soi_imgs = normalize.analyze_images(stack_dir, lower_thresh)
soi_imgs = normalize.images_used(soi_imgs, norm_tiffs, stack_tiffs)

# Visualise Image Histograms and Distribution

#### Normalization Signal (NS)

In [ ]:
nbins = 10
histograms = np.zeros((len(stack_tiffs[zmin:zmax]), nbins))
for idx, img in enumerate(norm_tiffs[zmin:zmax]):
    tiff = normalize.thresh(img, lower_thresh, upper_thresh, 1)
    hist, bins = np.histogram(tiff, bins = nbins)
    histograms[idx] = hist
histograms[histograms == 0] = 1
bin_labels = ["{}-{}".format(round(bins[i-1],1), round(bins[i],1)) for i in range(1, len(bins))]
hwt.plot_histograms(histograms, f"{base_dir} {stack_name} z{zmin}-{zmax} NS Heatmap of Histograms",
os.path.join(out_dir, f"{base_dir}_{stack_name}_z{zmin}-{zmax}_NS_Histograms.pdf"), bin_labels, save_figs)

#### Signal of Interest (SOI)

In [ ]:
nbins = 10
histograms = np.zeros((len(stack_tiffs[zmin:zmax]), nbins))
for idx, img in enumerate(stack_tiffs[zmin:zmax]):
    tiff = normalize.thresh(img, lower_thresh, upper_thresh, 1)
    hist, bins = np.histogram(tiff, bins = nbins)
    histograms[idx] = hist
histograms[histograms == 0] = 1
bin_labels = ["{}-{}".format(round(bins[i-1],1), round(bins[i],1)) for i in range(1, len(bins))]
hwt.plot_histograms(histograms, f"{base_dir} {stack_name} z{zmin}-{zmax} SOI Heatmap of Histograms",
os.path.join(out_dir, f"{base_dir}_{stack_name}_z{zmin}-{zmax}_SOI_Histograms.pdf"), bin_labels, save_figs)

In [ ]:
norm_means = np.array([])
norm_medians = np.array([])
norm_mins = np.array([])
norm_maxs = np.array([])
norm_vars = np.array([])
for img in norm_tiffs[zmin:zmax]:
    means, medians, maxs, mins, vars = normalize.tiff_dist(img, lower_thresh, upper_thresh, 1)
    norm_means = np.append(norm_means, means)
    norm_medians = np.append(norm_medians, medians)
    norm_mins = np.append(norm_mins, mins)
    norm_maxs = np.append(norm_maxs, maxs)
    norm_vars = np.append(norm_vars, vars)
hwt.plot_dist(norm_means, norm_medians, norm_mins, norm_maxs, norm_vars,
             f"{base_dir} {stack_name} z{zmin}-{zmax} NS",
             os.path.join(out_dir, f"{base_dir}_{stack_name}_z{zmin}-{zmax}_NS_min_max.pdf"), save_figs)

In [ ]:
stack_means = np.array([])
stack_medians = np.array([])
stack_mins = np.array([])
stack_maxs = np.array([])
stack_vars = np.array([])
for img in stack_tiffs[zmin:zmax]:
    means, medians, maxs, mins, vars = normalize.tiff_dist(img, lower_thresh, upper_thresh, 1)
    stack_means = np.append(stack_means, means)
    stack_medians = np.append(stack_medians, medians)
    stack_mins = np.append(stack_mins, mins)
    stack_maxs = np.append(stack_maxs, maxs)
    stack_vars = np.append(stack_vars, vars)
hwt.plot_dist(stack_means, stack_medians, stack_mins, stack_maxs, stack_vars,
             f"{base_dir} {stack_name} z{zmin}-{zmax} SOI",
             os.path.join(out_dir, f"{base_dir}_{stack_name}_z{zmin}-{zmax}_SOI_min_max.pdf"), save_figs)

#### SOI Depth Loss

In [ ]:
stack_means = np.array([])
for img in stack_tiffs[zmin:zmax]:
    mean = normalize.tiff_stats_thresh(img, lower_thresh, upper_thresh, 1)
    stack_means = np.append(stack_means, mean)
hwt.plot_MSI(stack_means,
             f"{base_dir} {stack_name} z{zmin}-{zmax}",
             os.path.join(out_dir, f"{base_dir}_{stack_name}_z{zmin}-{zmax}_SOI_Intensity.pdf"), save_figs)

### NS Depth Loss

In [ ]:
norm_means = np.array([])
for norm_tiff in norm_tiffs[zmin:zmax]:
    mean = normalize.tiff_stats_thresh(norm_tiff, lower_thresh, upper_thresh, 1)
    norm_means = np.append(norm_means, mean)
hwt.plot_MSI(norm_means,
             f"{base_dir} {norm_name} z{zmin}-{zmax}",
             os.path.join(out_dir, f"{base_dir}_{stack_name}_z{zmin}-{zmax}_NS_Intensity.pdf"), save_figs)

#### SOI and NS Depth Loss

In [ ]:
hwt.plot_MSI_soi_ns(soi, ns, stack_means, norm_means, f"{base_dir} {stack_name} {norm_name} z{zmin}-{zmax}_",
             os.path.join(out_dir, f"{base_dir}_{stack_name}n{norm_name}_z{zmin}-{zmax}_SOI_vs_NS.pdf"), save_figs)

# Normalization

##### Generating Normalized Images

In [ ]:
norm_stack_tiffs = normalize.mean_normalizer(stack_tiffs[zmin:zmax], norm_tiffs[zmin:zmax], lower_thresh, upper_thresh, True, False)

#### Z-Stack Distribution Post-Normalization

In [ ]:
norm_stack_means = np.array([])
for norm_stack_tiff in norm_stack_tiffs:
    mean = normalize.tiff_stats_thresh(norm_stack_tiff, 0, 255, 1)
    norm_stack_means = np.append(norm_stack_means, mean)
hwt.plot_MSI(norm_stack_means,
             f"{base_dir} {stack_name} norm {norm_name} z{zmin}-{zmax}_",
             os.path.join(out_dir, f"{base_dir}_{stack_name}n{norm_name}_z{zmin}-{zmax}_Normalized_SOI_Intensity.pdf"), save_figs)

In [ ]:
hwt.plot_MSI_grouped(stack_means, norm_means, norm_stack_means, f"{base_dir} {stack_name} norm {norm_name} z{zmin}-{zmax}_Grouped",
                os.path.join(out_dir, f"{base_dir}_{stack_name}n{norm_name}_z{zmin}-{zmax}_Normalized_Intensity_grouped.pdf"), save_figs)

In [ ]:
nbins = 10
histograms = np.zeros((len(norm_stack_tiffs), nbins))
for idx, img in enumerate(norm_stack_tiffs):
    tiff = normalize.thresh(img, 0, 255, 1)
    hist, bins = np.histogram(tiff, bins = nbins)
    histograms[idx] = hist
histograms[histograms == 0] = 1
bin_labels = ["{}-{}".format(round(bins[i-1],1), round(bins[i],1)) for i in range(1, len(bins))]
hwt.plot_histograms(histograms, f"{base_dir} {stack_name} z{zmin}-{zmax} Normalized SOI Heatmap of Histograms",
os.path.join(out_dir, f"{base_dir}_{stack_name}_z{zmin}-{zmax}_Adjusted_Normalized_SOI_Histograms.pdf"), bin_labels, True)

In [ ]:
tools.stack_gui(norm_stack_tiffs, norm_tiffs)

#### Heatmap Prior to Normalization

In [ ]:
imgs = norm_heatmap.stack(stack_tiffs[zmin:zmax], ['z', 'x', 'y'], z_multiplier)

In [ ]:
hwt.plot_MIP(imgs, f"{base_dir} {stack_name} z{zmin}-{zmax}", 
                os.path.join(out_dir, f"{base_dir}_{stack_name}_z{zmin}-{zmax}_HeatMap.pdf"), save_figs)

#### Heatmap After Normalization

In [ ]:
imgs = norm_heatmap.stack(norm_stack_tiffs, ['z', 'x', 'y'], z_multiplier)

In [ ]:
hwt.plot_MIP(imgs, f"{base_dir} {stack_name} norm {norm_name} z{zmin}-{zmax}", 
                os.path.join(out_dir, f"{base_dir}_{stack_name}n{stack_name}_z{zmin}-{zmax}_Normalized_HeatMap.pdf"), save_figs)